# 动画
要求：
- 保留Markdown格式
- 保留代码、URL和HTML标签
- 使用标准技术术语
- 仅输出翻译，不添加解释

翻译：
到目前为止，在本教程中我们一次只处理一个时间步长。然而，我们也讨论了静止卫星数据的主要优势之一是其时间分辨率。它可以快速为我们提供同一地点的许多图像。让我们通过制作GOES-16 ABI中尺度数据的MPEG4视频来探索这一特性。由于时间和处理所需的时间限制，我们将仅使用一小时内的1分钟数据（2018年5月11日21:00Z至2018年5月11日22:00Z）。

In [ ]:
%run ../init_notebook.py
from glob import glob
filenames = glob('../data/abi_l1b/20180511_texas_fire_abi_l1b_meso/*s201813121*.nc')
len(filenames)

通常，如果我们想要处理960个输入文件（ABI的60个时间步），就需要对每16个文件进行一次排序，然后为每个文件创建`Scene`对象。Satpy提供了一个名为`MultiScene`的辅助类，用于帮助完成此类及其他对多个`Scene`对象进行操作的任务。创建`MultiScene`的一种方法是使用其`from_files`类方法。这将根据时间和其他属性对文件进行排序，以确保每个时间步都有自己的Scene，并能够尽可能地进行操作。
需要注意的是，为了提高性能，`MultiScene` 内部使用 "生成器" 进行操作。生成器可能是一个复杂的主题，但基本上这意味着，类似于 dask 数组，我们不会在请求时对每个 `Scene` 执行操作。相反，我们会构建一系列待执行的操作，并在需要时执行它们。因此，建议不要将 MultiScenes 用于 "探索" 数据，通常在这种情况下你会访问 Scene 的许多属性。
让我们创建自己的 `MultiScene` 并了解一些基本操作。

In [ ]:
from satpy import MultiScene
mscn = MultiScene.from_files(filenames, reader='abi_l1b')

像一个正常的 `Scene`，`MultiScene` 也有一个 `load` 方法。让我们从上一课加载我们的 RGB。

In [ ]:
mscn.load(['my_rgb'])

要求：
- 保留Markdown格式
- 保留代码、URL和HTML标签
- 使用标准技术术语
- 仅输出翻译，不添加解释

要翻译的文本：
根据我们之前与单个 `Scene` 对象的工作经验，我们知道我们的RGB需要进行重采样。我们可以调用 `MultiScene.resample` 方法来获取一个新的 `MultiScene` 对象。

In [ ]:
new_mscn = mscn.resample(resampler='native')

最后，我们可以将已加载并重新采样的图像序列保存为MPEG4视频。我们将指定视频的文件名以及所需的帧每秒（FPS）数量。在底层，此方法使用`imageio` Python库和`ffmpeg`，将我们的图像序列制作成视频或GIF。
注意：根据您可用的硬件，处理60帧可能需要几分钟。

In [ ]:
%%time
new_mscn.save_animation('{name}_{start_time:%Y%m%d_%H%M%S}.mp4', fps=12)

要查看电影文件，请使用系统的文件浏览器导航到 `notebooks` 目录，然后使用能够播放它的应用程序（如 Quicktime, VLC 等）打开 `.mp4` 视频文件。
# 堆叠
`MultiScene` 还可用于可视化极轨卫星的多个轨道。让我们以一个简单的例子来说明，使用两个 VIIRS 过境数据，一个来自 NOAA-20，另一个来自 Suomi-NPP，并将它们堆叠在一起。此处使用的 `from_files` 方法会检测到我们拥有 NOAA-20 和 Suomi-NPP 数据，并按平台对文件进行分组。最终得到两个可用于操作的 Scene 对象。
由于重采样这些数据所需的时间，我们每个轨道将只处理两个数据粒。

In [ ]:
filenames = glob('../data/viirs_sdr/20180511_texas_fire_viirs_sdr/*j01*t203[13]*.h5') + glob('../data/viirs_sdr/20180511_texas_fire_viirs_sdr/*npp*.h5')
len(filenames)

In [ ]:
mscn = MultiScene.from_files(filenames, reader='viirs_sdr')
mscn.load(['I03'])

我们需要对数据进行重采样，以生成一个网格区域，从而显示我们选择的两个轨道。我们将使用 pyresample 的 `create_area_def` 函数，就像我们在重采样课程中所做的那样。

In [ ]:
from pyresample import create_area_def
my_area = create_area_def('my_area', {'proj': 'lcc', 'lon_0': -95, 'lat_0': 25, 'lat_1': 35},
                          width=3000, height=3000,
                          area_extent=[-110, 20, -85, 50], units='degrees')
my_area

In [ ]:
new_mscn = mscn.resample(my_area)

现在我们可以使用 `MultiScene.blend` 方法将各个 Scene 合并为一个新的 Scene。默认情况下，此方法会将每个 orbit 堆叠在下一个之上。

In [ ]:
blended_scn = new_mscn.blend()

现在我们有一个正常的 `Scene` 对象，这是我们习惯使用的。让我们绘制它。
注意：这可能需要几分钟才能完成。

In [ ]:
%%time
%matplotlib notebook

from dask.diagnostics import ProgressBar
with ProgressBar():
    blended_scn['I03'].plot.imshow()

有了更多的数据和时间，我们可以使用相同的代码来合并多个轨道，或生成极轨卫星数据的全球合成数据。